In [11]:
from IPython.display import display
import pandas as pd
import requests
from bs4 import BeautifulSoup
from info import urls

In [12]:
class Crawler:
    def __init__(self, base_url):
        self.base_url = base_url

    def get_html(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return None

    def fetch_posts(self):
        html = self.get_html(self.base_url)
        soup = BeautifulSoup(html, 'html.parser')
        return soup

In [13]:
class Parser:
    def __init__(self, soup):
        self.soup = soup
        self.posts = []

    def parse_post_list(self, base_url):
        rows = self.soup.find_all("tr")
        for row in rows:
            number = row.find('td', class_="b-num-box")
            title_box = row.find('div', class_="b-title-box")
            date = row.find('span', class_="b-date")
            writer = row.find('span', class_="b-writer")
            views = row.find('span', class_="hit")
            post_data = {'번호': number.get_text(strip=True) if number else '',
                         '제목': title_box.a.get_text(strip=True) if title_box and title_box.a else '',
                         "작성자": writer.get_text(strip=True) if writer else '',
                         '날짜': date.get_text(strip=True) if writer else '',
                         "조회수": views.get_text(strip=True) if views else '',
                         "본문링크": base_url + title_box.a['href'] if title_box and title_box.a else ''}
            self.posts.append(post_data)

    def parse_contents(self, post, content_soup):
        content = content_soup.find("div", class_="fr-view")
        post["내용"] = content.get_text(strip=True).strip() if content else ''

    def parse_images(self, post, content_soup):
        div = content_soup.find("div", class_="fr-view")
        img_tags = div.find_all('img')
        if img_tags:
            img_srcs = [img['src'] if 'src' in img.attrs and "https://homepage.cnu.ac.kr" in img['src'] else 'https://ai.cnu.ac.kr' + img['src'] for img in img_tags]
            post["이미지"] = ''.join(img_srcs) if len(img_srcs) == 1 else img_srcs
        else:
            post["이미지"] = None


In [14]:
class DataCleaner:
    def __init__(self, posts):
        self.posts = posts

    def clean_data(self):
        if self.posts:
            self.posts.pop(0)
        for post in self.posts:
            view = post["조회수"].replace("조회수", "").strip()
            post["조회수"] = view

In [15]:
class TableParser:
    def __init__(self, base_url):
        self.base_url = base_url

    def parse_html_table(self, table):
        rows = table.find_all("tr")
        table_data = []
        columns = []
        rowspan_dict = {}

        for i, row in enumerate(rows):
            tds = row.find_all(["td", "th"])
            row_data = []
            col_idx = 0
            while col_idx in rowspan_dict:
                if rowspan_dict[col_idx][1] > 0:
                    row_data.append(rowspan_dict[col_idx][0])
                    rowspan_dict[col_idx][1] -= 1
                    if rowspan_dict[col_idx][1] == 0:
                        del rowspan_dict[col_idx]
                    col_idx += 1
                else:
                    del rowspan_dict[col_idx]

            for td in tds:
                spans = td.find_all("span")
                ps = td.find_all("p")
                if spans:
                    cell_text = " ".join([span.get_text(strip=True) for span in spans])
                elif ps:
                    cell_text = " ".join([p.get_text(strip=True) for p in ps])
                else:
                    cell_text = td.get_text(strip=True)
                colspan = int(td.get("colspan", 1))
                rowspan = int(td.get("rowspan", 1))

                for _ in range(colspan):
                    row_data.append(cell_text)
                    if rowspan > 1:
                        rowspan_dict[col_idx] = [cell_text, rowspan - 1]
                    col_idx += 1

            if i == 0:
                columns = row_data
            else:
                while len(row_data) < len(columns):
                    row_data.append("")
                table_data.append(row_data)

        # Check if table_data and columns have the correct format and if columns contain meaningful data
        if len(columns) > 1 and all(len(row) == len(columns) for row in table_data) and len(table_data) > 0:
            # Check if columns contain actual text
            if any(column.strip() for column in columns):
                return pd.DataFrame(table_data, columns=columns)

        return None

    def find_valid_tables(self, soup):
        div = soup.find("div", class_="fr-view")
        if not div:
            return []

        tables = div.find_all("table")
        valid_tables = []
        seen_tables = set()

        for table in tables:
            nested_tables = table.find_all("table")
            if nested_tables:
                for nested_table in nested_tables:
                    table_str = str(nested_table)
                    if table_str not in seen_tables:
                        seen_tables.add(table_str)
                        df = self.parse_html_table(nested_table)
                        if df is not None:
                            valid_tables.append(df)
            else:
                table_str = str(table)
                if table_str not in seen_tables:
                    seen_tables.add(table_str)
                    df = self.parse_html_table(table)
                    if df is not None:
                        valid_tables.append(df)

        return valid_tables

    def table_main(self, post_url):
        soup = BeautifulSoup(Crawler(self.base_url).get_html(post_url), 'html.parser')
        tables = self.find_valid_tables(soup)
        dfs = []
        for table in tables:
            pd.set_option('display.max_colwidth', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_columns', None)
            dfs.append(table)
        return dfs

In [16]:
class NoticeCrawler:
    def __init__(self, base_url):
        self.base_url = base_url
        self.crawler = Crawler(base_url)
        self.soup = self.crawler.fetch_posts()
        self.parser = Parser(self.soup)
        self.cleaner = DataCleaner(self.parser.posts)
        self.table_parser = TableParser(base_url)

    def crawl(self):
        self.parser.parse_post_list(self.base_url)
        self.cleaner.clean_data()
        self.parse_contents_and_images()
        self.parse_tables()

    def parse_contents_and_images(self):
        for post in self.parser.posts:
            post_url = post["본문링크"]
            content_soup = BeautifulSoup(self.crawler.get_html(post_url), 'html.parser')
            self.parser.parse_contents(post, content_soup)
            self.parser.parse_images(post, content_soup)

    def parse_tables(self):
        for post in self.parser.posts:
            post_url = post["본문링크"]
            dfs = self.table_parser.table_main(post_url)
            if dfs:
                post["is_table"] = 1
                post["table"] = dfs
            else:
                post["is_table"] = 0
                post["table"] = None

    def display_tables(self):
        for post in self.parser.posts:
            if post["is_table"] == 1:
                for df in post["table"]:
                    display(df)

In [21]:
if __name__ == "__main__":
    url = urls["학사공지"]
    crawler = NoticeCrawler(url)
    crawler.crawl()
    for post in crawler.parser.posts:
        post_without_table = {k: v for k, v in post.items() if k != "table"}
        print(post_without_table)
    crawler.display_tables()

{'번호': '공지', '제목': '[★★★ 1학년 교과목 수강신청 안내 (본인이 직접 수강신청) ★★★]', '작성자': '조교 이희정', '날짜': '24.07.23', '조회수': '589', '본문링크': 'https://ai.cnu.ac.kr/ai/board/notice.do?mode=view&articleNo=519785&article.offset=0&articleLimit=10', '내용': '1학년 수강신청 날짜 및 시간을 꼭 확인 하시고 아래 안내 사항 확인 후본인이 직접 수강신청하시기 바랍니다. (2학기부터는 학과에서 수강신청 해주지 않음)[1학년 수강 신청 교과목]이수구분교과목명교과목번호분반학점졸업요건수강신청 기간교양미적분학11151-7008003필수1학년 수강신청 기간8. 6.(화) 15:00～17:00인공지능과 미래사회1004-7003053필수전공선형대수1214-1003003필수어드벤처디자인1215-1002003필수컴퓨터프로그래밍21216-1002003필수본인이 교양에서1개 교과목 선택해서 수강신청*3예비 수강신청 기간7.29(월)~7.31(수) 09:00~18:00-본인이 교양에서1개 교과목 선택해서 수강신청*:한 학기 수강 학점은 총18학점으로 학과에서 지정한 교과목(15학점)외에나머지3학점은 교양 교과목에서 본인이 희망하는 교과목으로 수강신청,될 수 있으면 졸업 필수 교과목인 핵심교양(목록은 붙임 졸업요건에서 확인)으로 수강 신청-인공지능과미래사회: 2022학번부터 지정된 전문기초교양으로2021학번은 인공지능개론(분반 상관없음.)수강 해야 함.-미적분학1: 2023학번부터 필수, 2021~2022학번은 선택(본인학번의 졸업요건 확인)[유의사항 및 일정]1.미적분학1,인공지능과미래사회:지정된 분반은 인공지능학과 학생을 위하여 확보된 분반이지만 타과 학생이 수강신청 할 경우 여석 확보가 어려울 수 있으므로가급적 빨리(8. 6.(화) 15:00정원 열리자마자)수강신청 하여 주시기 바랍니다.2.예비수강신청: 7.29(월) ~ 7.31.(수

,이수 구분,교과목명,교과목번호,분반,학점,졸업 요건,수강신청 기간
0,교양,미적분학 1,1151-7008,00,3,필수,1 학년 수강신청 기간 8. 6.( 화 ) 15:00 ～ 17:00
1,교양,인공지능과 미래사회,1004-7003,05,3,필수,
2,전공,선형대수,1214-1003,00,3,필수,
3,전공,어드벤처디자인,1215-1002,00,3,필수,
4,전공,컴퓨터프로그래밍 2,1216-1002,00,3,필수,
5,본인이 교양에서 1 개 교과목 선택해서 수강신청 *,본인이 교양에서 1 개 교과목 선택해서 수강신청 *,본인이 교양에서 1 개 교과목 선택해서 수강신청 *,본인이 교양에서 1 개 교과목 선택해서 수강신청 *,3,,예비 수강신청 기간 7.29( 월 )~7.31( 수 ) 09:00~18:00


,교과목명,실습기간,인정학점,
0,"백마인턴십 Ⅲ-1, Ⅲ-2, Ⅲ-3, Ⅲ-4",4주,3학점,
1,"백마인턴십 Ⅰ-1, Ⅰ-2, Ⅰ-3",8주,6학점,
2,백마인턴십II,1학기,18학점,


,교과목명,개설 학년-학기
0,어드벤처디자인,1-2
1,기초프로젝트랩,"2-1, 2-2"
2,심화프로젝트랩,"3-1, 3-2"
3,종합설계1,4-1
4,종합설계2,4-2


,구분,명칭,장학금액 *,선발기준,직전학기 평균평점,인원,비고,대학 추천 **
0,성적 장학금,성적우수,등록금 전액,성적우수자 및 학과 기준 적용,3.25 이상,총장이 정하는 인원,교내우수,ㅇ
1,성적 장학금,격려,등록금 일부 C급,성적우수자,2.25 이상,〃,교내격려,ㅇ
2,복지 장학금,백마,등록금 일부,"국가장학금 기준 준용 ※ 단, 국가 2유형 대상자는 등록금의 15%(예정) 지원 ※ 이연자( 등록유효복학자 ), 수업연한초과자는 선발대상에서 제외",좌동,적격자,,
3,복지 장학금,영탑A,등록금 전액,장애의 정도가 심한 장애인 (기존 장애1급～장애3급 학생),2.75 이상,〃,,ㅇ
4,복지 장학금,영탑B,등록금 일부 A급,장애의 정도가 심하지 않은 장애인 (기존 장애4급～장애6급 학생),2.75 이상,〃,,ㅇ
5,특별 장학금,체육특기자,등록금 전액,체육진흥원 추천자,1.75 이상,적격자,,ㅇ
6,특별 장학금,복수학위,등록금 전액 (4개 학기),국제교류본부장 추천자,3.50 이상 (전 학기 평점),〃,,
7,특별 장학금,외국어성적우수,등록금 일부 A급,"TOEIC, TOEFL 성적우수자(ETS 주관) •TOEIC 900점 이상 •TOEFL iBT 102점 이상 ※ 자격취득일부터 1년 이내 성적, 재학 중 1회*만 수혜 가능 * 단, 본교 출신 편입생이 동 장학금을 기수혜 한 경우에도, 편입 후 1회 수혜 가능. 편입생의 경우, 편입 후 취득한 성적에 한함 [ ʹ23. 8. 1.～ ʹ24 7. 31. 응시일자만 해당 ] ※ 단, 휴학학기에 등록금을 기납부한 복학생 (등록유효복학생) 은 선발대상에서 제외 ※ 수업일수 1/3선 (’24. 10. 11.) 이내 신청에 한해 지원",2.75 이상,〃,,ㅇ
8,특별 장학금,외국인 (6.25 유엔참전용사 후손),등록금전액 정규학기,6.25 유엔참전용사 후손(직계비속)임이 확인된 외국인 유학생(국제교류본부 추천자) ※ 해당자가 있는 경우 각 대학에서는 국제교류본부로 추천,2.50이상,〃,,
9,특별 장학금,외국인(학부),등록금 일부 B급,외국인 유학생 ※ 국적이 대한민국인 경우 제외,2.75 이상,〃,,ㅇ


,장학명,제출서류,제출기한,제출장소
0,영탑,"장학금신청서, 장애인증명서",상단 내용 확인 (시간 엄수),학과 사무실
1,외국어성적우수,재학중 1회이므로 신중하게 신청.,,
2,"보훈, 새터민, 학석사연계과정","장학금신청서 [단, 보훈,새터민 신규자는 대학수업료등 면제대상자증명서(교육지원대상자증명서) 추가 제출]",,


,과목명 (과목번호),분반,수강신청 대상,비고
0,컴퓨터과학적사고 (1004-7002),00~03,전계열,"(권장계열 : 이공계열) 이론(콘텐츠 활용) + 실습(실시간 원격, 파이썬기초)"
1,인공지능과 미래사회 (1004-7003),00~02,전계열,콘텐츠 활용 원격 강의 시간표 미지정
2,인공지능과 미래사회 (1004-7003),03~05,"컴퓨터융합학부 , 인공지능학과 전용 분반","컴퓨터융합학부 , 인공지능학과 전용 분반"
3,인공지능융합기초 (1004-7004),00~02,전계열,대면 강의
4,파이썬 프로그래밍 (1004-7005),00~01,전계열,이론(콘텐츠 활용) + 실습(실시간 원격)
5,파이썬 프로그래밍 (1004-7005),02,전계열,대면 강의
6,데이터분석 입문과활용 (1004-7006),00,전계열,이론(콘텐츠 활용) + 실습(실시간 원격)
7,컴퓨터 이해와 활용 (1004-7007),00,전계열,(권장계열 : 인문/사회/예체능) 대면 강의
8,컴퓨터 이해와 활용 (1004-7007),01~03,전계열,"(권장계열 : 인문/사회/예체능) 콘텐츠 활용 원격 강의, 시간표 미지정"
9,컴퓨터 이해와 활용 (1004-7007),04,전계열,콘텐츠 활용 원격 강의 시간표 미지정(외국인 전용 분반)


In [30]:
def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

def fetch_posts(base_url):
    html = get_html(base_url)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

url = urls["학사공지"]
soup = fetch_posts("https://ai.cnu.ac.kr/ai/board/notice.do?mode=view&articleNo=503557&article.offset=0&articleLimit=10")

def parse_files(content_soup):
    total_file = []
    file_box = content_soup.find("div", class_="b-file-box")
    lis = file_box.find_all("li") if file_box else total_file == None
    if lis:
        for file_num, li in enumerate(lis):
            file_dict = {}
            file = li.find_all("a")
            file_dict["다운로드"] = "https://ai.cnu.ac.kr/ai/board/notice.do?" + file[0]["href"]
            file_dict["미리보기"] = "https://ai.cnu.ac.kr" + file[1]["href"]
            total_file.append(file_dict)
    return total_file

parse_files(soup)

[]

In [ ]:
'''if lis:
    for file_num, li in enumerate(lis):
        file_dict = {}
        file = li.find_all("a")
        file_dict["다운로드"] = "https://ai.cnu.ac.kr" + file[0]["href"]
        file_dict["미리보기"] = "https://ai.cnu.ac.kr" + file[1]["href"]
        total_file.append(file_dict)
if len(total_file) >= 2 or len(total_file) == 0:
    post["첨부파일"] = total_file
elif len(total_file) == 1:
    post["첨부파일"] = ''.join(str(d) for d in total_file)'''